<a href="https://colab.research.google.com/github/ronenbendavid/IDC_NLP/blob/master/Ronen_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
In this assignment you will be creating tools for learning and testing language models.
The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.


*As a preparation for this task, place the data files somewhere in your drive so that you can access the files from this notebook. The files are available to download from the Moodle assignment activity*

The relevant files are:


*   en.csv (or the equivalent JSON file)
*   es.csv (or the equivalent JSON file)
*   fr.csv (or the equivalent JSON file)
*   in.csv (or the equivalent JSON file)
*   it.csv (or the equivalent JSON file)
*   nl.csv (or the equivalent JSON file)
*   pt.csv (or the equivalent JSON file)
*   tl.csv (or the equivalent JSON file)





**Part 1**

Write a function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. **Our token definition is a single UTF-8 encoded character**. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
from pathlib import Path
from glob import glob
import csv
import json
import pandas as pd
import os
os.chdir('/content/drive/My Drive/IDC/NLP/Assignment1/')
def preprocess():
  # TODO
 vocabulary = set()
 pathlist = glob('*.csv')
 for path in pathlist:
    with open(path, encoding='utf-8', newline='') as csv_file:
        csv_reader = pd.read_csv(csv_file, delimiter=',')
        line_count = 0
        for text in csv_reader['tweet_text']:
            list_text = list(text)
            vocabulary.update(list_text)
            line_count += 1
        #print(line_count)
 return sorted(list(vocabulary))

vocabulary = preprocess()
print(len(vocabulary))
print(vocabulary)
  

1859
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x80', '\x91', '\x92', '\x97', '\x9d', '¡', '£', '¤', '¥', '§', '¨', '©', 'ª', '«', '\xad', '®', '¯', '°', '²', '³', '´', '¶', '·', '¸', 'º', '»', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Å', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', '×', 'Ù', 'Ú', 'Ü', 'à', 'á', 'â', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', 'ø', 'ù', 'ú', 'û', 'ü', 'ė', 'Ğ', 'ğ', 'İ', 'ı', 'ń', 'ō', 'Œ', 'œ', 'Ş', 'ş', 'Š', 'Ÿ', 'ƒ', 'ʔ', 'ʕ', 'ʖ', 'ʰ', 'ʳ', '

**Part 2**

Write a function lm that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant n-1 sequences, and the values are dictionaries with the n_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{
  "ab":{"c":0.5, "b":0.25, "d":0.25},
  "ca":{"a":0.2, "b":0.7, "d":0.1}
}

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [8]:
import nltk.data
import nltk.tokenize 
from collections import Counter
from collections import defaultdict
def find_ngrams(text, n):
    counter = Counter()
    ngram = {}

    num_tokens, tokens = split_into_tokens(text.lower())
    for token in tokens:
        sequence  = {}
        for i,_ in enumerate(token):
            if(i>=n - 1):
                word, char = token[i - (n - 1):i], token[i]
                nchar_word = ''.join(word) + char
                counter[nchar_word] = counter.get(nchar_word, 0) + 1
                sequence[char] = counter[nchar_word]
                ngram[word] = sequence
                #print({word:ngram[word]})
    return ngram
def token_to_unigram(token):
    token = token.strip().strip(",.!|&-_()[]<>{}/\"'").strip()

    def has_no_chars(token):
        for char in token:
            if char.isalpha():
                return False
        return True

    if len(token) == 1 or token.isdigit() or has_no_chars(token):
        return None
    return token
def split_into_tokens(text):
    tokens = []
    for token in nltk.tokenize.WhitespaceTokenizer().tokenize(text):
    #for token in tokenize(text):
        unigram = token_to_unigram(token)
        if unigram:
            tokens.append(unigram)
    return len(tokens), tokens
def lm(n, vocabulary, data_file_path, add_one):
  # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
  # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
  # data_file_path - the data_file from which we record probabilities for our model
  # add_one - True/False (use add_one smoothing or not)

  # TODO
  voc_size = len(vocabulary)
  if add_one:
      model = defaultdict(lambda: defaultdict(lambda: 1/voc_size))
  else:
      model = defaultdict(lambda: defaultdict(lambda: 1e-08))
  with open(data_file_path, encoding='utf-8', newline='') as csv_file:
    csv_reader = pd.read_csv(csv_file, delimiter=',')
    corpus = []
    for tweet_text in csv_reader['tweet_text']:
            corpus.append(tweet_text)
    data = find_ngrams(" ".join(corpus),n)

    for word,chars in data.items():
        temp_dict = {}
        words_total = float(sum(chars.values()))
        for char, count in chars.items():
            if (add_one == True):
                p = float(count + 1)/(words_total + len(vocabulary))
            else:
                p = float(count)/words_total
            temp_dict[char] = p
        model[word] = temp_dict
  return model
n =3
file_name = 'en.csv'
model = lm(n, vocabulary,file_name , False)
print({k: v for i, (k, v) in enumerate(model.items()) if i < 50})
#print(sum(v for v in model['abc'].values()))
print(model['rt'])
print({k: v for i, (k, v) in enumerate(model.items()) if i < 50})
# for j, key in enumerate(model.keys()):
#     if j < 50:
#         print("ngram:{:s} {:f}".format(key,sum(model[key].values())))  

{'@h': {'i': 0.06206896551724138, 'l': 0.02666666666666667, 'a': 0.09793103448275862, 'r': 0.1103448275862069, 'y': 0.0910344827586207, 'c': 0.027126436781609194, 'n': 0.17471264367816092, 't': 0.2381609195402299, 'o': 0.0993103448275862, ':': 0.07264367816091954}, 'ha': {'u': 0.023801967629324024, 's': 0.18946366232941922, 't': 0.25261821643922566, 'i': 0.18406854966677244, 'n': 0.06569343065693431, '_': 0.00856870834655665, 'h': 0.0050777530942557915, 'a': 0.008251348778165662, 'l': 0.05553792446842272, 'p': 0.013329101872421454, 'e': 0.008886067914947636, 'r': 0.17613456045699777, ':': 0.00856870834655665}, 'an': {'d': 1.0}, 'nn': {'v': 0.025537634408602152, 'e': 0.29973118279569894, 'b': 0.008064516129032258, 'n': 0.14381720430107528, 't': 0.5161290322580645, 'x': 0.004032258064516129, ':': 0.002688172043010753}, 'na': {'n': 0.10649819494584838, 'a': 0.10613718411552346, 'l': 0.5790613718411552, 'y': 0.20830324909747291}, 'ah': {'h': 0.11515151515151516, 'c': 0.19393939393939394, '

**Part 3**

Write a function *eval* that returns the perplexity of a model (dictionary) running over a given data file.

In [10]:
import numpy as np
def compute_entropy_xi(ngram):
    ll = 0
    for key in ngram.items():
        p = key[1]
        ll += p*np.log2(p)
    return ll

def eval(n, model, data_file):
    """
    Compute the perplexity of ngrams from model
    """
    h_x = 0
    N = 0
    with open(data_file, encoding='utf-8', newline='') as csv_file:
        csv_reader = pd.read_csv(csv_file, delimiter=',')
        corpus = []
        for tweet_text in csv_reader['tweet_text']:
                corpus.append(tweet_text)
        data = find_ngrams(" ".join(corpus),n)
        N = len(model)   # Total number of words
        for word,chars in data.items():
            if word in model:
                ngram = model[word]
                #for i, ngram in enumerate(model.items()):
                h_x += compute_entropy_xi(ngram)
            #N += len(h_X)
    return pow(2, (-1.0 / N) * h_x)

n =3
file_name = 'en.csv'
model = lm(n, vocabulary,file_name , False)
test_file_name = 'fr.csv'
preplexity = eval(n,model,test_file_name)
print("Preplexity evaluation : {:f}".format(preplexity))
# import math
# from collections import defaultdict
# def lm_b(n, vocabulary, data_file_path, add_one):
#   # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
#   # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
#   # data_file_path - the data_file from which we record probabilities for our model
#   # add_one - True/False (use add_one smoothing or not)

#   voc_size = len(vocabulary)
#   if add_one:
#       model = defaultdict(lambda: defaultdict(lambda: 1/voc_size))
#   else:
#       model = defaultdict(lambda: defaultdict(lambda: 1e-08))
#   data_file = pd.read_csv(data_file_path)
#   for data in data_file['tweet_text'].values:
#       data = ["<s>"] + list(data) + ["</s>"]
#       for i in range(len(data) - n):
#           word, char = ''.join(data[i:i + n]), data[i + n]
#           model[''.join(word)][char] = model[word].get(char, 0) + 1

#   for word in model:
#       if add_one:
#           total_count = float(sum(model[word].values())) + voc_size
#           for char in model[word]:
#               model[word][char] = (model[word][char] + 1) / total_count
#       else:
#           total_count = float(sum(model[word].values()))
#           for char in model[word]:
#               model[word][char] /= total_count

#   return model

# def eval_b(n, model, data_file_path):
#   # n - the n-gram that you used to build your model (must be the same number)
#   # model - the dictionary (model) to use for calculating perplexity
#   # data_file - the tweets file that you wish to claculate a perplexity score for

#   data_file = pd.read_csv(data_file_path)
#   count = 0
#   total = 0.0
#   for data in data_file['tweet_text'].values:
#       data = ['<s>'] + list(data) + ['</s>']
#       for i in range(len(data) - n):
#           count += 1
#           word, char = ''.join(data[i:i + n]), data[i + n]
#           total += math.log2(model[word][char])

#   ent = -1 / count * total
#   per = 2 ** ent

#   return per
# n =3
# file_name = '/content/drive/My Drive/IDC/NLP/Assignment1/en.csv'
# model_b = lm_b(n, vocabulary,file_name , False)
# print(eval_b(2, model_b, '/content/drive/My Drive/IDC/NLP/Assignment1/en.csv'))



Preplexity evaluation : 2.589641


**Part 4**

Write a function *match* that creates a model for every relevant language, using a specific value of *n* and *add_one*. Then, calculate the perplexity of all possible pairs (e.g., en model applied on the data files en ,es, fr, in, it, nl, pt, tl; es model applied on the data files en, es...). This function should return a pandas DataFrame with columns [en ,es, fr, in, it, nl, pt, tl] and every row should be labeled with one of the languages. Then, the values are the relevant perplexity values.

In [11]:
# def match(n, add_one):
#   # n - the n-gram to use for creating n-gram models
#   # add_one - use add_one smoothing or not

#   #TODO
#   file_path = '/content/drive/My Drive/IDC/NLP/Assignment1/{}.csv'
#   vocabulary = preprocess()

#   lang = ['en' ,'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']
#   df = pd.DataFrame(columns=lang, index=lang)
#   for l1 in lang:
#     l1_model = lm(n, vocabulary, file_path.format(l1), add_one)
#     for l2 in lang:
#       df.at[l1, l2] = eval(n, l1_model, file_path.format(l2))

#   return df
def match(n, add_one):
#  path_name = ''
#  csvglob = path_name + '*.csv'
 pathlist = glob('*.csv')
 num = len(pathlist)
 matrix = np.empty(shape=(num,num),dtype='float')
 headers_list = []
 i = 0
 for model_path in pathlist:
    head, tail = os.path.split(model_path)
    headers_list.append(tail.rstrip('.csv'))
    j = 0
    modeli = lm(n, vocabulary,model_path, add_one)
    for data_file_path in pathlist:
        #if ( i != j ):
        pp = eval(n,modeli,data_file_path)
        matrix[i][j] = pp
        #print ("Preplexity of model {:d} with data file {:d} is {:f}".format(i,j,pp))
        j = j+1
    i = i + 1
#  print(matrix)
 dataframe = pd.DataFrame(matrix,columns=headers_list,index=headers_list)
 print("passed dataframe")
 return dataframe

print (match(2, False))

passed dataframe
          en        tl        pt  ...        in        it        nl
en  3.688392  1.817754  1.705147  ...  1.836229  1.728866  1.855658
tl  1.756077  3.526284  1.580283  ...  1.851085  1.622713  1.772741
pt  2.077481  1.913563  3.775446  ...  1.933758  1.710553  1.882190
e   2.051986  1.956943  1.904216  ...  1.896964  1.928765  1.774035
fr  1.783147  1.876383  1.593590  ...  1.907068  1.643849  1.756990
in  1.776085  1.769760  1.573840  ...  3.460067  1.549607  1.733575
it  2.241573  2.214109  1.836097  ...  2.108697  3.714469  2.024682
nl  2.145320  2.159224  1.770823  ...  2.064769  1.790204  3.916633

[8 rows x 8 columns]


**Part 5**

Run match with *n* values 1-4, once with add_one and once without, and print the 8 tables to this notebook, one after another.

In [0]:
max_n = 4 
for n in range(4): 
  print("n = {:d} add_one:{:b}".format(n+1,False))
  print(match(n+1,False)) 
  print("n = {:d} add_one:{:b}".format(n+1,True))
  print(match(n+1,True))

n = 1 add_one:0
         en       es       fr       in       it       nl       pt       tl
en  11.1978  11.1978  11.1978  11.1978  11.1978  11.1978  11.1978  11.1978
es  10.2632  10.2632  10.2632  10.2632  10.2632  10.2632  10.2632  10.2632
fr  11.2849  11.2849  11.2849  11.2849  11.2849  11.2849  11.2849  11.2849
in  8.04565  8.04565  8.04565  8.04565  8.04565  8.04565  8.04565  8.04565
it  10.7234  10.7234  10.7234  10.7234  10.7234  10.7234  10.7234  10.7234
nl  2.91596  2.91596  2.91596  2.91596  2.91596  2.91596  2.91596  2.91596
pt  3.86417  3.86417  3.86417  3.86417  3.86417  3.86417  3.86417  3.86417
tl  4.75737  4.75737  4.75737  4.75737  4.75737  4.75737  4.75737  4.75737
n = 1 add_one:1
         en       es       fr       in       it       nl       pt       tl
en  11.1272  11.1272  11.1272  11.1272  11.1272  11.1272  11.1272  11.1272
es  10.1974  10.1974  10.1974  10.1974  10.1974  10.1974  10.1974  10.1974
fr  11.2112  11.2112  11.2112  11.2112  11.2112  11.2112  11.2112  1

# **Good luck!**
max_n = 4
for n = 1 in range(max_n):
  data = match(n,False)
  print(data)
  data = match(n,True)
  print(data)
  